In [73]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [42]:
# Create SparkSession
spark = SparkSession.builder.master("local[*]") \
                    .appName('test') \
                    .getOrCreate()

In [43]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv')

In [44]:
df.head(5)

[Row(dispatching_base_num='B00009', pickup_datetime='2019-10-01 00:23:00', dropOff_datetime='2019-10-01 00:35:00', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00013', pickup_datetime='2019-10-01 00:11:29', dropOff_datetime='2019-10-01 00:13:22', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_number='B00013'),
 Row(dispatching_base_num='B00014', pickup_datetime='2019-10-01 00:11:43', dropOff_datetime='2019-10-01 00:37:20', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_number='B00014'),
 Row(dispatching_base_num='B00014', pickup_datetime='2019-10-01 00:56:29', dropOff_datetime='2019-10-01 00:57:47', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_number='B00014'),
 Row(dispatching_base_num='B00014', pickup_datetime='2019-10-01 00:23:09', dropOff_datetime='2019-10-01 00:28:27', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_num

In [45]:
import pandas as pd
df_pandas = pd.read_csv('fhv_tripdata_2019-10.csv')

In [46]:
df_pandas.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [47]:
spark.createDataFrame(df_pandas).schema

KeyboardInterrupt: 

In [ ]:
from pyspark.sql import types

In [ ]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PUlocationID', types.IntegerType(), True), 
    types.StructField('DOlocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [48]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')

In [53]:
df['pickup_datetime'].date

TypeError: 'Column' object is not callable

In [52]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [29]:
df = df.repartition(6)

In [30]:
df.write.parquet('fhv/2019/10/')

In [31]:
df = spark.read.parquet('fhv/2019/10/')

In [ ]:
#  df = Seq(("2019-07-01 12:01:19.000"),
#     ("2019-06-24 12:01:19.000"),
#     ("2019-11-16 16:44:55.406"),
#     ("2019-11-16 16:50:59.406")).toDF("input_timestamp")


#   df.withColumn("datetype",
#     to_date(col("input_timestamp"),"yyyy-MM-dd"))
#     .show(false)


#   df.withColumn("ts",to_timestamp(col("input_timestamp")))
#     .withColumn("datetype",to_date(col("ts")))
#     .show(false)


#   df.withColumn("ts",to_timestamp(col("input_timestamp")))
#     .withColumn("datetype",col("ts").cast(DateType))
#     .show(false)

In [62]:
from pyspark.sql.functions import to_date 

# df.withColumn("datetype",
#     to_date(col("input_timestamp"),"yyyy-MM-dd")).show()

In [33]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [70]:
df = df.withColumn('pickup_date', to_date(df.pickup_datetime))
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|pickup_date|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009| 2019-10-01|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013| 2019-10-01|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014| 2019-10-01|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014| 2019-10-01|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264| 

In [72]:
df.select('pickup_datetime','dropOff_datetime','PUlocationID','DOlocationID', to_date(df.pickup_datetime).alias('pickup_date')) \
.filter(df['pickup_date'] == '2019-10-15') \
.count()

62610

In [ ]:
df.select('pickup_datetime','dropOff_datetime','PUlocationID','DOlocationID') \
.filter(df.pickup_datetime=='B01413') \
.count()

In [88]:
df2 = df.withColumn('ride_duration_in_hrs',(unix_timestamp('dropOff_datetime') - unix_timestamp('pickup_datetime'))/3600)

In [89]:
df2.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+--------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|pickup_date|ride_duration_in_hrs|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+--------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009| 2019-10-01|                 0.2|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013| 2019-10-01| 0.03138888888888889|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014| 2019-10-01| 0.42694444444444446|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|       

In [90]:
df2.select(max(df2.ride_duration_in_hrs).alias('longest ride')).show()

+------------+
|longest ride|
+------------+
|    631152.5|
+------------+



In [87]:
maxHrs = df2.select(max(df2.ride_duration_in_hrs).alias('longest ride')).show()
df2.select('pickup_datetime', 'dropOff_datetime','ride_duration_in_hrs').filter(df2.ride_duration_in_hrs == maxHrs).show()

+------------+
|longest ride|
+------------+
|  2272149000|
+------------+

+---------------+----------------+--------------------+
|pickup_datetime|dropOff_datetime|ride_duration_in_hrs|
+---------------+----------------+--------------------+
+---------------+----------------+--------------------+

